In [1]:
import pandas as pd
import random
import os
import gc

import matplotlib.pyplot as plt

import numpy as np

from bokeh.palettes import Spectral10
from bokeh.io import output_notebook, show

output_notebook()
                                                                                                                                                                                                                                                                                                                                                        
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

palette = Spectral10

pd.set_option('display.float_format', lambda x: '%.3f' % x)

import bokeh
bokeh.__version__

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.dao import csv_dao #load_user_gps_csv, load_user_gps_time_window, load_user_stop_regions_centroids
from src.dao.dbdao import DBDAO
from src.utils import math
from src.utils import time_utils
from src.utils import geo
#from src.plot.plot import add_centroid_figure, add_calculated_centroid_figure
from src.plot import plot
#from src.utils.geo import index_clusters
from src.plot import plot
from src.dao import places_dao
from bokeh.layouts import gridplot

Loading BokehJS ...

working dir /home/tales/dev/master/mdc_analysis


In [2]:
def load_users_gps_data(userids, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    df = pd.DataFrame()
    for userid in userids:
        user_gps_df = load_user_gps_csv(userid)
        user_gps_df["userid"] = [userid] * len(user_gps_df)
        df = df.append(user_gps_df)
        
    df = local_time(df)
    
    if cols != "*":
        df = df[cols]
    
    df = df.sort_values("local_time")

    return df

def load_user_gps_data(userid, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    user_gps_data = load_user_gps_csv(userid)
    user_gps_data["userid"] = [userid] * len(user_gps_data)
    
    if cols != "*":
        user_gps_data = user_gps_data[cols]
        
    user_gps_data = user_gps_data.sort_values("local_time")
    
    return user_gps_data

In [3]:
user_ids = [6085, 5965] + DBDAO().users_with_places().sample(3).tolist()
user_ids

[6085, 5965, 5989, 5950, 5964]

In [4]:
plot_counter = 1

In [5]:
def plot_visit(visit_locations, color="navy", title="user visits", limit_points=500, p=None, size=8, legend=None, width=800, height=600):
    if len(visit_locations) > limit_points:
        visit_locations = visit_locations.sample(limit_points)
    p = plot.plot_user_loc(user_data=visit_locations[["latitude", "longitude"]].drop_duplicates(), 
                      title=title, width=width, height=height, size=size, color=color, p=p, 
                      legend=legend, alpha=0.3)
    return p

## Match Stop Region with WORK points (from table Places)

In [6]:
def plot_points_and_centroids(points, centroids, user_id, width=600, height=400):
    p = plot.plot_stop_regions_centroids(centroids, title="User: {} - GPS points and Stop Region".format(str(user_id)), width=width, height=height)
    p = plot.plot_user_loc(user_data=points, p=p)
    return p

In [7]:
user_id = 5963
print("User: {}".format(user_id))
work_points = places_dao.places_work(user_id, do_remove_outliers=True)
work_sr = places_dao.load_stop_regions_work(user_id, verbose=True)["work"]

p = plot_points_and_centroids(work_points, work_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User: 5963
614 stop regions

40 stop regions WORK
574 stop regions NOT WORK


Plot # 1


In [8]:
user_id = 6014
print("User: {}".format(user_id))
work_points = places_dao.places_work(user_id, do_remove_outliers=True)
work_sr = places_dao.load_stop_regions_work(user_id, verbose=True)["work"]

p = plot_points_and_centroids(work_points, work_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User: 6014
637 stop regions

20 stop regions WORK
617 stop regions NOT WORK


Plot # 2


In [9]:
user_id = 5943
print("User: {}".format(user_id))

work_points = places_dao.places_work(user_id, do_remove_outliers=True)
work_sr = places_dao.load_stop_regions_work(user_id, verbose=True)["work"]

p = plot_points_and_centroids(work_points, work_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User: 5943
251 stop regions

18 stop regions WORK
233 stop regions NOT WORK


Plot # 3


## Plotting WORK and NOT WORK Stop Regions

In [10]:
def plot_work_and_others(user_id):
    print("User {}".format(user_id))
    
    sr = work_sr = places_dao.load_stop_regions_work(user_id, verbose=True)
    
    work_sr = sr["work"]
    not_work_sr = sr["not_work"]
        
    p = plot.plot_stop_regions_centroids(not_work_sr, legend="NOT work", fill_color="navy", width=600, height=400)
    p = plot.plot_stop_regions_centroids(work_sr, legend="work", p=p, fill_color="magenta", width=600, height=400)

    return p

#### <font color="blue"> Please apply ZOOM to see Work Stop Regions distribution better </font>

In [11]:
p = plot_work_and_others(5963)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5963
614 stop regions

40 stop regions WORK
574 stop regions NOT WORK


Plot # 4


In [12]:
p = plot_work_and_others(6187)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6187
187 stop regions

11 stop regions WORK
176 stop regions NOT WORK


Plot # 5


In [13]:
p = plot_work_and_others(6188)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6188
821 stop regions

5 stop regions WORK
816 stop regions NOT WORK


Plot # 6


In [14]:
p = plot_work_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6075
322 stop regions

23 stop regions WORK
299 stop regions NOT WORK


Plot # 7


In [15]:
p = plot_work_and_others(6038)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6038
647 stop regions

13 stop regions WORK
634 stop regions NOT WORK


Plot # 8


<font color="blue"> It seems that there are lots of Stop Regions very close to <b>Work</b> Stop Regions but not marked as Work.
<br>
They were not marked as Work because the time that the person stayed there was not registered in <i>Places</i> table.
<br>
These Stop Regions will be considered Work.
</font>

# Considering Stop Regions close to Work also as Work
# Checking also time inferred work

In [16]:
def infer_work(userid):
    user_stop_region_centroids = csv_dao.load_user_stop_regions_centroids(userid)

    local_start_time = time_utils.enrich_time_columns(user_stop_region_centroids, time_col="local_start_time")[["hour", "weekday"]]
    local_end_time = time_utils.enrich_time_columns(user_stop_region_centroids, time_col="local_end_time")[["hour", "weekday"]]

    return user_stop_region_centroids[ (local_start_time["weekday"] == local_end_time["weekday"]) &
                                       (local_start_time["weekday"] != "Saturday") &
                                       (local_start_time["weekday"] != "Sunday") &
                                      
                                       (
                                           (
                                                (local_start_time["hour"] >= 9) &
                                                (local_end_time["hour"] <= 10)
                                           )
                                           |
                                           (
                                                (local_start_time["hour"] >= 15) &
                                                (local_end_time["hour"] <= 16)
                                           )
                                       ) ]

def plot_work_expanded_and_others(user_id, radius=20):
    print("User {}".format(user_id))
    sr = places_dao.load_stop_regions_work(user_id, verbose=True)

    work_sr = sr["work"]
    not_work_sr = sr["not_work"]
    
    work_close_sr = geo.infer_close_sr_as_work(work_sr, not_work_sr, radius)
    work_inferred_sr = infer_work(user_id)[["latitude", "local_end_time", "local_start_time", "longitude", "sr_id", "tag", "user_id"]]

    print(len(work_close_sr), "close to work < {}m".format(radius))
    print(len(work_inferred_sr), "time inferred".format(radius))
    
    p = plot.plot_stop_regions_centroids(work_inferred_sr, legend="work by time inferrence", fill_color="green", width=600, height=400, mark="square")
    p = plot.plot_stop_regions_centroids(not_work_sr, legend="NOT work", p=p, fill_color="navy", width=600, height=400)
    p = plot.plot_stop_regions_centroids(work_close_sr, legend="work close < {}m".format(radius), p=p, fill_color="#ff9a00", width=600, height=400)
    p = plot.plot_stop_regions_centroids(work_sr, legend="work", p=p, fill_color="magenta", width=600, height=400)
    
    return p

#### <font color="blue"> Please apply ZOOM to see WORK Stop Regions distribution better </font>

In [17]:
p = plot_work_expanded_and_others(5963)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5963
614 stop regions

40 stop regions WORK
574 stop regions NOT WORK
46 close to work < 20m
62 time inferred


Plot # 9


In [18]:
p = plot_work_expanded_and_others(6188)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6188
821 stop regions

5 stop regions WORK
816 stop regions NOT WORK
17 close to work < 20m
98 time inferred


Plot # 10


In [19]:
p = plot_work_expanded_and_others(6038)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6038
647 stop regions

13 stop regions WORK
634 stop regions NOT WORK
106 close to work < 20m
61 time inferred


Plot # 11


In [20]:
p = plot_work_expanded_and_others(6085)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6085
1398 stop regions

68 stop regions WORK
1330 stop regions NOT WORK
133 close to work < 20m
110 time inferred


Plot # 12


In [21]:
p = plot_work_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6053
328 stop regions

29 stop regions WORK
299 stop regions NOT WORK
23 close to work < 20m
28 time inferred


Plot # 13


In [22]:
p = plot_work_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5986
530 stop regions

8 stop regions WORK
522 stop regions NOT WORK
14 close to work < 20m
45 time inferred


Plot # 14


In [23]:
p = plot_work_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6082
543 stop regions

26 stop regions WORK
517 stop regions NOT WORK
101 close to work < 20m
57 time inferred


Plot # 15


In [24]:
p = plot_work_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5989
689 stop regions

12 stop regions WORK
677 stop regions NOT WORK
200 close to work < 20m
88 time inferred


Plot # 16


Work place by time inferrence shows many Stop Regions. They seem to have a huge dispersion over the map (the green square marks).

## Saving WORK inferred for all users

In [25]:
# for user_id in DBDAO().users_with_places():
#     print()
#     print("---------------")
#     print("user_id: {}".format(user_id))
    
#     dirpath = "outputs/work_inferred/"
#     filename = "work_stop_regions_user_{}_v2.csv".format(user_id)
    
#     if os.path.exists(dirpath + filename):
#         print("User already computed")
#         continue
    
#     sr = places_dao.load_stop_regions_work(user_id, verbose=True)

#     work_sr = sr["work"]
#     not_work_sr = sr["not_work"]
#     work_close_sr = geo.infer_close_sr_as_work(work_sr, not_work_sr, radius=20)
    
#     print("{} stop regions close to work".format(len(work_close_sr)))
    
#     work_sr = sr["work"].append(work_close_sr)
#     work_sr["sr_id"].rename("sr_id").to_csv(dirpath + filename, header=True, index=False)